In [76]:
import numpy as np
from scipy.optimize import minimize, differential_evolution

In [58]:
# создание произвольного нормального вектора размерности 4 -> далее
# из него через функцию generate_complex_vector можно сделать вектор размерности 2 из комплексных чисел
def generateState():
    s0 = np.random.random_sample() * 2 - 1
    s1 = np.random.random_sample() * 2 - 1
    s2 = np.random.random_sample() * 2 - 1
    s3 = np.random.random_sample() * 2 - 1
    state = np.array([s0, s1, s2, s3]/np.sqrt(s0 ** 2 + s1 ** 2 + s2 ** 2 + s3 ** 2))
    return state

In [59]:
generateState()

array([ 0.48825859, -0.41595997, -0.65200517, -0.40431437])

In [5]:
password = 'gaiphi7aan'

process_index = 0
number_of_states = 100

# вещественная основа для входного вектора
state = np.array([1, 0, 0, 0])

# вещественная основа для базиса в котором проводим измерение
projector = np.array([1, 0, 0, 0])

In [15]:
# функция для создания комплексного вектора из вещественной основы
def generate_complex_vector(state):
    return( state[0]+state[1]*1j,  state[2]+state[3]*1j )

In [18]:
# входной вектор (комплексные числа)
alpha, beta = generate_complex_vector(state) 

# комплексный базис измерения
m, n = generate_complex_vector(projector)

In [92]:
# расчет теоритической вероятности 
def probability(a, b, phi, alpha, beta, m, n):
    part1 = np.abs(m) ** 2 * np.abs((a * alpha + b * beta)) ** 2
    part2 = np.conj(m) * n * (a*alpha + b*beta) * np.exp(-phi*1j) * (a*np.conj(beta) - alpha*np.conj(b))
    part3 = m * np.conj(n) * (np.conj(a)*np.conj(alpha) + np.conj(b)*np.conj(beta)) * np.exp(phi*1j) * (np.conj(a)*beta - np.conj(b)*alpha)
    part4 = np.abs(n) ** 2 * np.abs(np.conj(a)*beta - alpha*np.conj(b))
    return part1 + part2 + part3 + part4

In [95]:
# функция расчёта ошибки
def loss_function(params):
    ar, ai, br , bi, phi = params
    N = len(result)
    mnk = 0
    for result_item in result:
        # получение величин измерения
        state = result_item[0]
        projector = result_item[1]
        p0_measured = result_item[2]
        
        # вычисление теоритической вероятности
        alpha, beta = generate_complex_vector(state) 
        m, n = generate_complex_vector(projector)
        a = ar + ai*1j
        b = br + bi*1j
        p0_theoretical = probability(a, b, phi, alpha, beta, m, n)
        mnk += pow((p0_theoretical - p0_measured),2)
    mnk = mnk/N
    return(mnk)

In [82]:
state = generateState()
projector = generateState()

# входной вектор (комплексные числа)
alpha, beta = generate_complex_vector(state) 

# комплексный базис измерения
m, n = generate_complex_vector(projector)

# Initial parameters
a = 1+1*1j
b = 0+1*1j
phi = 0

probability(a, b, phi, alpha, beta, m, n)

(0.42982244976568035-0.17347998234841602j)

In [67]:
# функция измерения состояния

def run_measurement(state, projector, process_index=0, number_of_states=100):
    import socket, sys

    #request = password + ' 1 100 0.5 0.5 0.5 0.5 1 0 0 0'

    request = password + ' ' + str(process_index)+ ' ' + str(number_of_states) + ' ' + \
    ' '.join([str(elem) for elem in np.append(state, projector)])


    host = 'ape.qotlabs.org'
    port = 2018

    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.settimeout(2)

    try:
        s.connect((host, port))
    except:
        print('Unable to connect')
        sys.exit()

    s.send(request.encode())
    data = s.recv(4096).decode()
    n = [int(item) for item in data.split()]
    return(n[0], n[1])

In [72]:
run_measurement(state, projector)

(48, 52)

In [ ]:
from tqdm import tqdm

In [73]:
%%time
result = []
for i in tqdm(range(0,100)):
    # генерируем вход. состояние
    state = generateState()
    projector = generateState()
    n = run_measurement(state, projector)
    p0 = n[0] / number_of_states
    result.append((state, projector, p0))
    
    

CPU times: user 108 ms, sys: 55.7 ms, total: 163 ms
Wall time: 37.5 s


In [83]:
result[0][0], result[0][1], result[0][2]

(array([ 0.76864003, -0.35390594, -0.52754922, -0.075066  ]),
 array([-0.54930157,  0.38435067, -0.05754459,  0.73975061]),
 0.46)

In [100]:
x0 = [0.5, 0.5, 0.5, 0.5, 1]
optimization_result = minimize(loss_function, x0, method=  )

/usr/lib/python3.6/site-packages/scipy/optimize/optimize.py:643: ComplexWarning: Casting complex values to real discards the imaginary part
  grad[k] = (f(*((xk + d,) + args)) - f0) / d[k]
/usr/lib/python3.6/site-packages/scipy/optimize/linesearch.py:171: ComplexWarning: Casting complex values to real discards the imaginary part
  amin, amax, isave, dsave)


In [101]:
optimization_result

      fun: 0.006827997440026877
 hess_inv: array([[ 2.48855424e+03,  4.80567131e+02, -6.95184083e+00,
        -4.37717614e+00, -5.48068881e+03],
       [ 4.80567131e+02,  9.35760403e+01, -9.50692694e-01,
        -1.11750130e+00, -1.05854853e+03],
       [-6.95184083e+00, -9.50692694e-01,  3.57113837e+00,
        -5.01213980e-01,  1.51504983e+01],
       [-4.37717614e+00, -1.11750130e+00, -5.01213980e-01,
         3.04418632e+00,  9.89027464e+00],
       [-5.48068881e+03, -1.05854853e+03,  1.51504983e+01,
         9.89027464e+00,  1.20767580e+04]])
      jac: array([ 2.58442014e-08,  3.21248081e-07, -2.57277861e-08,  2.42143869e-08,
        1.46683306e-08])
  message: 'Optimization terminated successfully.'
     nfev: 560
      nit: 67
     njev: 80
   status: 0
  success: True
        x: array([-1.75589707e-01,  9.06194662e-01, -3.55382106e-02,  3.12071306e-04,
        3.51495145e+00])

In [102]:
[round(item, 3) for item in optimization_result.x]

[-0.176, 0.906, -0.036, 0.0, 3.515]

In [105]:
import pickle

In [121]:
with open('dump5.pkl','rb') as file:
    result5 = pickle.load(file)

In [122]:
result = result5

In [129]:
x0 = [0.1, 0.3, 0.4, 0.2, 1]
optimization_result = minimize(loss_function, x0, method= )

/usr/lib/python3.6/site-packages/scipy/optimize/optimize.py:643: ComplexWarning: Casting complex values to real discards the imaginary part
  grad[k] = (f(*((xk + d,) + args)) - f0) / d[k]
/usr/lib/python3.6/site-packages/scipy/optimize/linesearch.py:171: ComplexWarning: Casting complex values to real discards the imaginary part
  amin, amax, isave, dsave)


In [130]:
optimization_result

      fun: 0.00034432667151237045
 hess_inv: array([[  6.0137323 ,   5.62207237,  -5.39188315,   4.40363901,
         19.16939685],
       [  5.62207237,   8.40302585,  -6.11337091,   7.32759126,
         24.83882574],
       [ -5.39188315,  -6.11337091,   8.0868996 ,  -6.95737218,
        -22.08780723],
       [  4.40363901,   7.32759126,  -6.95737218,  10.50218027,
         25.09951322],
       [ 19.16939685,  24.83882574, -22.08780723,  25.09951322,
         89.45632652]])
      jac: array([-1.38740870e-06,  5.74002479e-06, -3.97110489e-06, -5.02910916e-06,
       -9.53623385e-07])
  message: 'Optimization terminated successfully.'
     nfev: 266
      nit: 33
     njev: 38
   status: 0
  success: True
        x: array([ 0.51010495, -0.42766266,  0.46588217,  0.501709  ,  2.33130021])

In [125]:
a = optimization_result.x[0] + optimization_result.x[1]*1j
b = optimization_result.x[2] + optimization_result.x[3]*1j
phi = optimization_result.x[4]

In [126]:
-np.exp(1j*phi)*np.conj(b)

(-0.04236580503008397-0.6833628933441832j)

In [127]:
np.exp(1j*phi)*np.conj(a)

(-0.6614413241976673+0.0747767906858906j)